In [21]:
# Optimising a portfolio using a Monte Carlo simulation with Efficient Frontier 
# to balance the return with the optimal risk for a client.
# Also, the volatility of stocks like NVDA will be calculated, the Sharpe Ratio,
# know what and how to make Monte Carlo Simulations, Efficient Frontier, 
# calculate Correlation, use Linear Regression, calculate the Beta and CAPM.

# 1) Use pandas and NumPy to calculate the Return of a Portfolio
# 2) Risk and Volatility of a stock calculated as Average True Range (ATR)
# 3) Combine Risk and Return into Sharpe Ratio
# 4) Use Monte Carlo Simulation to optimize portfolio with Risk and Return
# 5) How to balance your portfolio with Correlation
# 6) Use Linear Regression to find how X causes Y
# 7) Measure a stock’s volatility in relation to the overall market as Beta.
# 8) CAPM – Relationship between systematic risk and expected return

## For analysis on an index
## import pandas_datareader.data as pdr
## import yfinance as yf
## yf.pdr_override()
## from datetime import datetime
## data = pdr.get_data_yahoo('^GSPC', datetime(1970,1,1))


# PART 1: PORTFOLIO RETURN

import pandas_datareader.data as pdr
import numpy as np
import datetime as dt
import yfinance as yfin
yfin.pdr_override()

# To get around a breaking change in the yahoo finance API

ticker = "NVDA"
start = dt.datetime(2019, 1, 1)
data = pdr.get_data_yahoo(ticker, start)

[*********************100%***********************]  1 of 1 completed


In [5]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,32.660000,34.619999,32.512501,34.055000,33.799728,50875200
2019-01-03,33.447498,33.790001,31.922501,31.997499,31.757648,70555200
2019-01-04,32.735001,34.432499,32.424999,34.047501,33.792286,58562000
2019-01-07,34.625000,36.222500,34.107498,35.849998,35.581272,70916000
2019-01-08,36.672501,36.695000,34.224998,34.957500,34.695465,78601600


In [6]:
data.index

DatetimeIndex(['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07',
               '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11',
               '2019-01-14', '2019-01-15',
               ...
               '2023-04-11', '2023-04-12', '2023-04-13', '2023-04-14',
               '2023-04-17', '2023-04-18', '2023-04-19', '2023-04-20',
               '2023-04-21', '2023-04-24'],
              dtype='datetime64[ns]', name='Date', length=1085, freq=None)

In [7]:
data.dtypes

Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [8]:
type(data)

pandas.core.frame.DataFrame

In [10]:
### Using NumPy
# Pandas dataframes are built upon NumPy data arrays. Numpy helps with more advanced financial 
# analysis.
# NumPy dataframes are more restrictive.

data.to_numpy()
# The names of the columns have been dropped.

array([[3.26599998e+01, 3.46199989e+01, 3.25125008e+01, 3.40550003e+01,
        3.37997284e+01, 5.08752000e+07],
       [3.34474983e+01, 3.37900009e+01, 3.19225006e+01, 3.19974995e+01,
        3.17576485e+01, 7.05552000e+07],
       [3.27350006e+01, 3.44324989e+01, 3.24249992e+01, 3.40475006e+01,
        3.37922859e+01, 5.85620000e+07],
       ...,
       [2.76709991e+02, 2.80299988e+02, 2.70000000e+02, 2.71040009e+02,
        2.71040009e+02, 4.27713000e+07],
       [2.69519989e+02, 2.71829987e+02, 2.67220001e+02, 2.71190002e+02,
        2.71190002e+02, 3.44766000e+07],
       [2.70130005e+02, 2.73660004e+02, 2.66709991e+02, 2.70420013e+02,
        2.70420013e+02, 3.38865000e+07]])

In [11]:
arr = data.to_numpy()

In [12]:
arr.shape
# Which means there are 1085 rows, and 6 columns

(1085, 6)

In [13]:
len(data)
# Same number of rows

1085

In [14]:
arr[0]

array([3.26599998e+01, 3.46199989e+01, 3.25125008e+01, 3.40550003e+01,
       3.37997284e+01, 5.08752000e+07])

In [15]:
data.head(1)

# Data in the NumPy array is the same, but in a different format, but with the loss of the column
# names and the formatting comes speed in computation.

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,32.66,34.619999,32.512501,34.055,33.799728,50875200


In [16]:
arr.dtype
# In NumPy data arrays, there is only 1 data type, a float. In a pandas dataframe, there can be a
# data type for each comlumn.
# This is how arrays are represented inside C programming language.

dtype('float64')

In [17]:
# Making a subset of the data
small = arr[:10, 0].copy()
# Accessing the first row of the array until index 10, which is row 9, and the first entry of each row

In [18]:
small

array([32.65999985, 33.44749832, 32.73500061, 34.625     , 36.67250061,
       35.47499847, 35.45000076, 36.08250046, 36.68000031, 37.93999863])

In [19]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,32.660000,34.619999,32.512501,34.055000,33.799728,50875200
2019-01-03,33.447498,33.790001,31.922501,31.997499,31.757648,70555200
2019-01-04,32.735001,34.432499,32.424999,34.047501,33.792286,58562000
2019-01-07,34.625000,36.222500,34.107498,35.849998,35.581272,70916000
2019-01-08,36.672501,36.695000,34.224998,34.957500,34.695465,78601600


In [22]:
np.max(small)
# Calling it from the NumPy library, and add the NumPy array and it finds the maximum value

37.939998626708984

In [23]:
small.max()
# Calling it directly on it

37.939998626708984

In [24]:
small.argmax()
# We know it is on index 9, which means it is on the 10th row

9

In [25]:
small[small.argmax()]

37.939998626708984

In [26]:
# Calculating the log return using NumPy on the first 10 rows of NVDA stock
np.log(small)


array([3.48615108, 3.509977  , 3.48844486, 3.54457596, 3.60202717,
       3.56882818, 3.56812327, 3.585808  , 3.60223166, 3.63600593])

In [27]:
np.log(data)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,3.486151,3.544432,3.481625,3.527977,3.520453,17.744886
2019-01-03,3.509977,3.520165,3.463311,3.465658,3.458134,18.071906
2019-01-04,3.488445,3.539001,3.478930,3.527757,3.520233,17.885597
2019-01-07,3.544576,3.589680,3.529517,3.579344,3.571819,18.077007
2019-01-08,3.602027,3.602640,3.532956,3.554133,3.546609,18.179903
...,...,...,...,...,...,...
2023-04-18,5.617970,5.638710,5.611558,5.622826,5.622826,17.917843
2023-04-19,5.611704,5.634790,5.606978,5.632322,5.632322,17.395300
2023-04-20,5.622970,5.635860,5.598422,5.602266,5.602266,17.571378


In [28]:
data/data.shift()
# Calculating the record divided by the previous record

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,1.024112,0.976025,0.981853,0.939583,0.939583,1.386829
2019-01-04,0.978698,1.019014,1.015741,1.064068,1.064068,0.830017
2019-01-07,1.057736,1.051986,1.051889,1.052941,1.052940,1.210956
2019-01-08,1.059134,1.013044,1.003445,0.975105,0.975105,1.108376
...,...,...,...,...,...,...
2023-04-18,1.036439,1.040880,1.034956,1.024628,1.024628,1.881389
2023-04-19,0.993753,0.996087,0.995431,1.009542,1.009542,0.593011
2023-04-20,1.011330,1.001071,0.991481,0.970391,0.970391,1.192531


In [29]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,32.660000,34.619999,32.512501,34.055000,33.799725,50875200
2019-01-03,33.447498,33.790001,31.922501,31.997499,31.757650,70555200
2019-01-04,32.735001,34.432499,32.424999,34.047501,33.792294,58562000
2019-01-07,34.625000,36.222500,34.107498,35.849998,35.581264,70916000
2019-01-08,36.672501,36.695000,34.224998,34.957500,34.695454,78601600


In [31]:
33.447498 / 32.660000

# This is equivalent to a percentage change, day after day.
# A percentage change of the logarythm is useful too

1.0241120024494796

In [34]:
np.log(data/data.shift())

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,0.023826,-0.024267,-0.018314,-0.062319,-0.062319,0.327020
2019-01-04,-0.021532,0.018836,0.015619,0.062099,0.062099,-0.186309
2019-01-07,0.056131,0.050680,0.050588,0.051587,0.051586,0.191410
2019-01-08,0.057451,0.012960,0.003439,-0.025210,-0.025211,0.102896
...,...,...,...,...,...,...
2023-04-18,0.035791,0.040066,0.034359,0.024330,0.024330,0.632010
2023-04-19,-0.006267,-0.003921,-0.004580,0.009497,0.009497,-0.522543
2023-04-20,0.011266,0.001071,-0.008556,-0.030056,-0.030056,0.176078


In [35]:
# Adding the percentages of the change of the logs 

np.sum(np.log(data/data.shift()))

Open         2.112752
High         2.067455
Low          2.104537
Close        2.071999
Adj Close    2.079524
Volume      -0.406359
dtype: float64

In [38]:
np.log(data/data.iloc[0]).tail(1)

# This is the same result
# The problem with normal percentage changes is that you cannot sum them up, whereas 
# with logarithmic changes, you can.

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-04-24,2.112752,2.067455,2.104537,2.071999,2.079524,-0.406359


In [41]:
small.shape
# Which means there are 10 values

(10,)

In [40]:
small.reshape(2, 5)

array([[32.65999985, 33.44749832, 32.73500061, 34.625     , 36.67250061],
       [35.47499847, 35.45000076, 36.08250046, 36.68000031, 37.93999863]])

In [42]:
# This is useful if you want the array to be reshaped as a long column

small.reshape(10, 1)

array([[32.65999985],
       [33.44749832],
       [32.73500061],
       [34.625     ],
       [36.67250061],
       [35.47499847],
       [35.45000076],
       [36.08250046],
       [36.68000031],
       [37.93999863]])

In [43]:
# This is the equivalent of 
small.reshape(-1, 1)

array([[32.65999985],
       [33.44749832],
       [32.73500061],
       [34.625     ],
       [36.67250061],
       [35.47499847],
       [35.45000076],
       [36.08250046],
       [36.68000031],
       [37.93999863]])

In [44]:
# PORTFOLIOS AND CALCULATING THE RETURN

tickers = ['NVDA', 'INTC', 'AMD', 'TSM']

start = dt.datetime(2020, 1, 1)

data = pdr.get_data_yahoo(tickers, start)

[*********************100%***********************]  4 of 4 completed


In [45]:
data.head()

Adj Close                                       Close             \
                  AMD       INTC       NVDA        TSM        AMD       INTC   
Date                                                                           
2020-01-02  49.099998  54.984596  59.760391  56.058464  49.099998  60.840000   
2020-01-03  48.599998  54.315811  58.803883  54.209766  48.599998  60.099998   
2020-01-06  48.389999  54.162170  59.050476  53.584198  48.389999  59.930000   
2020-01-07  48.250000  53.258411  59.765381  54.452515  48.250000  58.930000   
2020-01-08  47.830002  53.294571  59.877476  54.854004  47.830002  58.970001   

                                       High             ...        Low  \
                 NVDA        TSM        AMD       INTC  ...       NVDA   
Date                                                    ...              
2020-01-02  59.977501  60.040001  49.250000  60.970001  ...  59.180000   
2020-01-03  59.017502  58.060001  49.389999  60.700001  ...  58.525002   
2020-01-06  59.264999  57.389999  48.860001  60.200001  ...  57.817501   
2020-01-07  59.982498  58.320000  49.389999  59.799999  ...  59.097500   
2020-01-08  60.095001  58.750000  48.299999  59.320000  ...  59.537498   

                            Open                                     Volume  \
                  TSM        AMD       INTC       NVDA        TSM       AMD   
Date                                                                          
2020-01-02  59.599998  46.860001  60.240002  59.687500  59.599998  80331100   
2020-01-03  58.040001  48.029999  59.810001  58.775002  58.970001  73127400   
2020-01-06  57.130001  48.020000  59.590000  58.080002  57.599998  47934900   
2020-01-07  56.740002  49.349998  59.779999  59.549999  57.450001  58061400   
2020-01-08  58.110001  47.849998  58.889999  59.939999  58.189999  53767000   

                                          
                INTC      NVDA       TSM  
Date                                      
2020-01-02  18056000  23753600   8432600  
2020-01-03  15293900  20538400  10546400  
2020-01-06  17755200  26263600   8897200  
2020-01-07  21876100  31485600   7444300  
2020-01-08  23133500  27710800   5381500  

[5 rows x 24 columns]

In [46]:
data = data['Adj Close']
data.head()

,AMD,INTC,NVDA,TSM
Date,,,,
2020-01-02,49.099998,54.984596,59.760391,56.058464
2020-01-03,48.599998,54.315811,58.803883,54.209766
2020-01-06,48.389999,54.162170,59.050476,53.584198
2020-01-07,48.250000,53.258411,59.765381,54.452515
2020-01-08,47.830002,53.294571,59.877476,54.854004


In [48]:
# Allocating a arbitrary percetnage to each stock in a portfolio

portfolios = [.25, .15, .40, .20]

In [49]:
np.sum(portfolios)

1.0

In [51]:
# Assuming $100,000 invested
# First, normalising the data

data/data.iloc[0]

,AMD,INTC,NVDA,TSM
Date,,,,
2020-01-02,1.000000,1.000000,1.000000,1.000000
2020-01-03,0.989817,0.987837,0.983994,0.967022
2020-01-06,0.985540,0.985043,0.988121,0.955863
2020-01-07,0.982688,0.968606,1.000083,0.971352
2020-01-08,0.974134,0.969264,1.001959,0.978514
...,...,...,...,...
2023-04-18,1.828513,0.578889,4.629655,1.572287
2023-04-19,1.831772,0.566522,4.673831,1.556054
2023-04-20,1.835234,0.561248,4.535446,1.592801


In [52]:
# AMD gained, INTC lost nearly half, NVDA increased 4.5 times

(data/data.iloc[0])*portfolios

,AMD,INTC,NVDA,TSM
Date,,,,
2020-01-02,0.250000,0.150000,0.400000,0.200000
2020-01-03,0.247454,0.148176,0.393598,0.193404
2020-01-06,0.246385,0.147756,0.395248,0.191173
2020-01-07,0.245672,0.145291,0.400033,0.194270
2020-01-08,0.243534,0.145390,0.400784,0.195703
...,...,...,...,...
2023-04-18,0.457128,0.086833,1.851862,0.314457
2023-04-19,0.457943,0.084978,1.869533,0.311211
2023-04-20,0.458809,0.084187,1.814178,0.318560


In [53]:
# CALCULATING THE ABSOLUTE DOLLAR RETURN OF A MODELLED PORTFOLIO WITH THEIR ALLOCATED WEIGHTINGS
(data/data.iloc[0])*portfolios*100000

# Modelling the portfolio with $100,000 now it has been normalised, you can see the dollar return, with
# the weightings of the portfolio

,AMD,INTC,NVDA,TSM
Date,,,,
2020-01-02,25000.000000,15000.000000,40000.000000,20000.000000
2020-01-03,24745.417507,14817.552967,39359.770834,19340.439417
2020-01-06,24638.493327,14775.639207,39524.825627,19117.255139
2020-01-07,24567.210539,14529.090428,40003.339753,19427.044808
2020-01-08,24353.362178,14538.954877,40078.369302,19570.284286
...,...,...,...,...
2023-04-18,45712.831756,8683.340997,185186.212947,31445.741828
2023-04-19,45794.300019,8497.834414,186953.259030,31121.082190
2023-04-20,45880.857134,8418.721619,181417.827388,31856.028319


In [55]:
weight = np.random.random(4)
weight

array([0.72822945, 0.31175492, 0.87749579, 0.10662877])

In [56]:
weight /= weight.sum()

In [57]:
weight.sum()

1.0

In [ ]:
# This is useful for computing a monte carlo simulation